In [4]:
import mysql.connector as mysql
from mysql.connector import Error
import sqlite3
import numpy as np
import pandas as pd

### DATA PREPROCESSING

In [3]:
# Nhận tất cả dữ liệu thô từ database

try:
    connection = mysql.connect(host='localhost',database='air_quality',user='root',password='cuong#Super2001')
    if connection.is_connected():
        cursor = connection.cursor()
        sql = "select * from air_quality_raw"
        cursor.execute(sql)
        record = cursor.fetchall()
        print('Kích thước dữ liệu: ',len(record))
        connection.commit()
        cursor.close()
except Error as e:
    print('Error',e)

Kích thước dữ liệu:  29531


In [5]:
# Chuyển dữ liệu sang dạng DataFrame
df = pd.DataFrame(record,columns=['ID','City','Date','PM2.5','PM10','NO','NO2','NOx','NH3','CO','SO2','O3','Benzene','Toluene','Xylene','AQI','AQI_Bucket'])
# df = pd.read_csv('IOT/city_day.csv')
df = df.drop(columns=['ID'])

In [6]:
'''
Loại bỏ 3 thuộc tính có chỉ số corr thấp nhất Xylene(0.07), Benzene(0.21), SO2(0.25) và các dữ liệu không phải dạng số
'''
df = df.drop(columns=['City','Date','AQI_Bucket','Xylene','Benzene','SO2'])
df.head()

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,O3,Toluene,AQI
0,NaN,NaN,0.92,18.22,17.15,NaN,0.92,133.36,0.02,NaN
1,NaN,NaN,0.97,15.69,16.46,NaN,0.97,34.06,5.50,NaN
2,NaN,NaN,17.40,19.30,29.70,NaN,17.40,30.70,16.40,NaN
3,NaN,NaN,1.70,18.48,17.97,NaN,1.70,36.08,10.14,NaN
4,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.31,18.89,NaN


In [8]:
# Bỏ các giá trị trống
df = df.dropna()
df.shape

(10960, 10)

### DATABASE CONNECTION

In [16]:
# Tạo database cho dữ liệu sau khi xử lý
try:
    connection = mysql.connect(host='localhost',database='air_quality',user='root',password='cuong#Super2001')
    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute('CREATE TABLE if not exists preprocessed_air_quality (ID int NOT NULL AUTO_INCREMENT PRIMARY KEY,`PM2.5` double,`PM10` double,`NO` double,`NO2` double,NOx double,`NH3` double,`CO` double,O3 double,Toluene double,AQI double)')
        print('New table was created')
        connection.commit()
        cursor.close()
except Error as e:
    print('Error',e)

New table was created


In [13]:
# Import records of corr_matrix into database
try:
    connection = mysql.connect(host='localhost',database='air_quality',user='root',password='cuong#Super2001')
    if connection.is_connected():
        cursor = connection.cursor()
        for i,row in df.iterrows():
            sql = "INSERT INTO preprocessed_air_quality(`PM2.5`,`PM10`,`NO`,`NO2`,NOx,`NH3`,`CO`,O3,Toluene,AQI) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql,tuple(row))
            # print('Inserted record')
        connection.commit()
        cursor.close()
except Error as e:
    print('Error',e)

In [15]:
# Nếu đã tồn tại, có thể xóa để tạo một database khác


try:
    connection = mysql.connect(host='localhost',database='air_quality',user='root',password='cuong#Super2001')
    if connection.is_connected():
        cursor = connection.cursor()
        tbl_name = "preprocessed_air_quality"
        sql = "drop table if exists "+tbl_name
        cursor.execute(sql)
        print('Table was dropped')
        connection.commit()
        cursor.close()
except Error as e:
    print('Error',e)


Table was dropped
